# *Chapter 20*<br> Heat Flow via Time Stepping 

| | | |
|:---:|:---:|:---:|
| ![image](Figs/Cover.png)|[From **COMPUTATIONAL PHYSICS**, 3rd Ed, 2015](http://physics.oregonstate.edu/~rubin/Books/CPbook/index.html) <br>RH Landau, MJ Paez, and CC Bordeianu (deceased) <br>Copyrights: <br> [Wiley-VCH, Berlin;](http://www.wiley-vch.de/publish/en/books/ISBN3-527-41315-4/) and [Wiley & Sons, New York](http://www.wiley.com/WileyCDA/WileyTitle/productCd-3527413154.html)<br>  R Landau, Oregon State Unv, <br>MJ Paez, Univ Antioquia,<br> C Bordeianu, Univ Bucharest, 2015.<br> Support by National Science Foundation.|![image](Figs/BackCover.png)|

>*As the present now will later be past<br>
>The order is rapidly fadin’<br>
>And the first one now will later be last<br>
>For the times they are a-changin’. *   - Bob Dylan

**20 Heat Flow via Time Stepping**<br>
[20.1 Heat Flow via Time-Stepping (Leapfrog)](#20.1)<br>
[20.2 The Parabolic Heat Equation (Theory)](#20.2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[20.2.1 Solution: Analytic Expansion](#20.2.1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[20.2.2 Solution: Time-Stepping](#20.2.2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[20.2.3 Von Neumann Stability Assessment](#20.2.3)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[20.2.4 Heat Equation Implementation](#20.2.4)<br>
[20.3 Assessment and Visualization](#20.3)<br>
[20.4 Improved Heat Flow: Crank-Nicolson Method](#20.4)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[20.4.1 Solution of Tridiagonal Matrix Equations](#20.4.1)<br>
 &nbsp;&nbsp;&nbsp;&nbsp;[20.4.2 Implementation, Assessment](#20.4.2)<br>
 
 *In this chapter examine the heat equation and develop the leapfrog
method for solving it on a space-time lattice. We also develop an
improved Crank-Nicolson method that determines the solution over all of
space in a single step. Time stepping is simple, yet important, and we
will see it again when we attack various wave equations.*

** This Chapter’s Lecture, Slide Web Links, Applets & Animations**

| | |
|---|---|
|[All Lectures](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/index.html)|[![anything](Figs/RHLlectureMod4.png)](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/index.html)|

|*Lecture (Flash)*| *Slides* | *Sections*|*Lecture (Flash)*| *Slides* | *Sections*|  
|- - -|:- - -:|:- - -:|- - -|:- - -:|:- - -:|
|[Intro to PDE’s](http://science.oregonstate.edu/~rubin/Books/CPbook/eBook/VideoLecs/PDE_Intro/PDE_Intro.html)|[pdf](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Slides/Slides_NoAnimate_pdf/PDE's_08Mar.pdf)| 17.1 | [PDE Electrostatics I](http://science.oregonstate.edu/~rubin/Books/CPbook/eBook/VideoLecs/Elec1/Elec1.html)|[pdf](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Slides/Slides_NoAnimate_pd/ElectricField_10Mar.pdf)| 17.2 | 
| [Electrostatics II](http://science.oregonstate.edu/~rubin/Books/CPbook/eBook/VideoLecs/Elec2/Elec2.html)|[pdf](http://science.oregonstate.edu/~rubin/Books/CPbook/eBook//Slides/Slides_NoAnimate_pdf/ElectricField_10Mar.pdf)| 17.4 |[Finite Elements Electrostatics](http://science.oregonstate.edu/~rubin/Books/CPbook/eBook/VideoLecs/FEM/FEM.html)|[pdf](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Slides/Slides_NoAnimate_pdf/FiniteElements_26May.pdf)| 17.10 | 
| [PDE Heat](http://science.oregonstate.edu/~rubin/Books/CPbook/eBook/VideoLecs/Heat/Heat.html)|[pdf](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Slides/Slides_NoAnimate_pdf/Heat_06Ap10.pdf)|17.16|[Heat Crank-N](http://science.oregonstate.edu/~rubin/Books/CPbook/eBook/VideoLecs/Heat_CN/Heat_CN.html)|[pdf](http://physics.oregonstate.edu/~rubin/Books/CPbook/eBook/Lectures/Slides/Slides_NoAnimate_pdf/HeatCrank_96Ap10.pdf)| 17.19|
|[Heat Equation Applet](http://science.oregonstate.edu/~rubin/Books/CPbook/eBook/Applets/index.html)| | | | |

# 20.1  Heat Flow via Time-Stepping (Leapfrog)  <a id="20.1"></a>

**Problem:** You are given an aluminum bar of length $L=1 \mbox{m}$ and
width $w$ aligned along the $x$ axis (Figure 20.1). It is insulated
along its length but not at its ends. Initially the bar is at a uniform
temperature of 100 C, and then both ends are placed in contact with ice
water at 0 C. Heat flows out of the noninsulated ends only. Your
**problem** is to determine how the temperature will vary as we move
along the length of the bar at later times.

![image](Figs/Fig20_1.png)

**Figure 20.1** A metallic bar insulated along its length with its ends in contact
with ice. The bar is colored solid red and the insulation is of lighter color.

# 20.2  The Parabolic Heat Equation (Theory) <a id="20.2"></a>

A basic fact of nature is that heat flows from hot to cold, that is,
from regions of high temperature to regions of low temperature. We give
these words mathematical expression by stating that the rate of heat
flow $\textbf{H}$ through a material is proportional to the gradient of
the temperature $T$ across the material:

$$\tag*{20.1}
\textbf{H} =- K \mathbf{\nabla} T(\mathbf{x}, t),$$

where $K$ is the thermal conductivity of the material. The total amount
of heat $Q(t)$ in the material at any one time is proportional to the
integral of the temperature over the material’s volume:

$$\tag*{20.2} Q(t) = \int d\textbf{x} C \rho(\textbf{x}) T(\textbf{x}, t),$$

where $C$ is the specific heat of the material and $\rho$ is its
density. Because energy is conserved, the rate of decrease in $Q$ with
time must equal the amount of heat flowing out of the material. After
this energy balance is struck and the divergence theorem applied, there
results the *heat equation*:

$$\tag*{20.3}
\frac{\partial T(\textbf{x}, t)}{\partial t} =  \frac{K}{C \rho}
\nabla^2 T(\textbf{x}, t).$$

The heat equation (20.3) is a parabolic PDE with space and time as
independent variables. The specification of this problem implies that
there is no temperature variation in directions perpendicular to the bar
($y$ and $z$), and so we have only one spatial coordinate in the
Laplacian:

$$\tag*{20.4}
\frac {\partial T(x,t)}{\partial t} = \frac{K}{C\rho}
\frac{\partial ^2 T(x,t)}{\partial x^2}.$$

As given, the initial temperature of the bar and the boundary conditions
are

$$\tag*{20.5} T(x,t=0) = 100 \mbox{C}, \quad T(x=0,t) = T(x=L,t) = 0
\mbox{C}.$$

### 20.2.1  Solution: Analytic Expansion<a id="20.2.1"></a>

Analogous to Laplace’s equation, the analytic solution starts with the
assumption that the solution separates into the product of functions of
space and time:

$$\tag*{20.6} T(x,t) = X(x) {\cal T}(t).$$

When (20.6) is substituted into the heat equation (20.4) and the
resulting equation is divided by $X(x) {\cal T}(t)$, two noncoupled
ODE’s result:

$$\tag*{20.7}
\frac{d^2 X(x)}{dx^2} +k^2X(x) =0, \qquad \frac{d {\cal
T}(t)}{dt}+k^2  \frac{C}{C\rho}  {\cal T}(t) = 0,$$

where $k$ is a constant still to be determined. The boundary condition
that the temperature equals zero at $x=0$ requires a sine function for
$X$:

$$\tag*{20.8} X(x)= A \sin k x.$$

The boundary condition that the temperature equals zero at $x=L$
requires the sine function to vanish there:

$$\tag*{20.9}
\sin k L  = 0 \ \Rightarrow \  k = k_n = n\pi/ L , \quad
n=1,2,\ldots.$$

To avoid blow up, the time function must be a decaying exponential with
$k$ in the exponent:

$$\tag*{20.10}
 {\cal T}(t) = e^{-k_{n}^2 t/C\rho}, \  \Rightarrow
\ T(x,t) = A_n \sin k_n x e^{-k_{n}^2
t/C\rho},$$

where $n$ can be any integer and $A_n$ is an arbitrary constant. Because
(20.4) is a linear equation, the most general solution is a linear
superposition of $X_n(x)T_n(t)$ products for all values of $n$:

$$\tag*{20.11} T(x,t)=\sum_{n=1}^\infty A_n \sin k_{n} x e^{-k_{n}^2
t/C\rho}.$$

The coefficients $A_n$ are determined by the initial condition that at
time $t=0$ the entire bar has temperature $T=100$ K:

$$T(x,t=0)= 100 \ \Rightarrow \ \sum_{n=1}^\infty A_n\sin k_{n} x
= 100. \tag*{20.12}$$

Projecting the sine functions determines $A_n = {4 T_0 }/{n\pi}$ for $n$
odd, and so

$$\tag*{20.13} T(x,t) = \sum_{n=1,3,\ldots}^\infty \frac{4 T_0}{n\pi} \;\sin k_n x
e^{-k_n^2 Kt/(C\rho)}.$$

### 20.2.2  Solution: Time-Stepping<a id="20.2.2"></a>

As we did with Laplace’s equation, the numerical solution is based on
converting the differential equation to a finite-difference
(“difference”) equation. We discretize space and time on a lattice
(Figure 20.2) and solve for solutions on the lattice sites. The sites
along the top with white centers correspond to the known values of the
temperature for the initial time, while the sites with white centers
along the sides correspond to the fixed temperature along the
boundaries. If we *also* knew the temperature for times along the bottom
row, then we could use a relaxation algorithm as we did for Laplace’s
equation. However, with only the top and side rows known, we shall end
up with an algorithm that steps forward in time one row at a time, as in
the children’s game *leapfrog*.

![image](Figs/Fig20_2.png)

**Figure 20.2** The algorithm for the heat equation in which the temperature
at the location $\textit{x}=\textit{i} \Delta \textit{x}$ and time
$\textit{t}=(\textit{j}+\text{1})\Delta t$ is computed from the temperature
values at three points of an earlier time. The nodes with white centers
correspond to known initial and boundary conditions. (The boundaries are
placed artificially close for illustrative purposes.)

As is often the case with PDE’s, the algorithm is customized for the
equation being solved and for the constraints imposed by the particular
set of initial and boundary conditions. With only one row of times to
start with, we use a forward-difference approximation for the time
derivative of the temperature:

$$\tag*{20.14}
\frac{\partial T(x,t)}{\partial t} \simeq \frac{T(x,t+\Delta t)-
T(x,t)}{\Delta t}.$$

Because we know the spatial variation of the temperature along the
entire top row and the left and right sides, we are less constrained
with the space derivative as with the time derivative. Consequently, as
we did with the Laplace equation, we use the more accurate
central-difference approximation for the space derivative:

$$\tag*{20.15}
\frac{\partial^2 T(x,t)}{\partial x^2} \simeq \frac{T(x +\Delta
x,t) +T(x-\Delta x,t)-2 T(x,t)}{(\Delta x)^2}.$$

Substitution of these approximations into (20.4) yields the heat
difference equation

$$\tag*{20.16}
\frac{T(x,t+ \Delta t)-T(x,t)}{\Delta t} = \frac{K}{C\rho}
\frac{T(x+\Delta x,t) +T(x-\Delta x,t) -2 T(x,t)}{\Delta
x^2}.$$

We reorder (20.16) into a form in which $T$ can be stepped forward in
$t$:

$$\tag*{20.17}
 T_{i,j+1} = T_{i,j}+ \eta
\left[T_{i+1,j}+T_{i-1,j}-2T_{i,j}\right], \quad \eta  = \frac{K
\Delta t}{C\rho \Delta x^2},$$

where $x=i \Delta x$ and $t = j \Delta t$. This algorithm is *explicit*
because it provides a solution in terms of known values of the
temperature. If we tried to solve for the temperature at all lattice
sites in Figure. 20.2 simultaneously, then we would have an *implicit*
algorithm that requires us to solve equations involving unknown values
of the temperature. We see that the temperature at space-time point
$(i,j+1)$ is computed from the three temperature values at an earlier
time $j$ and at adjacent space values $i\pm 1, i$. We start the solution
at the top row, moving it forward in time for as long as we want and
keeping the temperature along the ends fixed at 0 K (Figure 20.3).

![image](Figs/Fig20_3.png) 

**Figure 20.3** A numerical calculation of the temperature
<span>*versus*</span> position and <span>*versus*</span> time, with
isotherm contours projected onto the horizontal plane.

### 20.2.3  Von Neumann Stability Assessment<a id="20.2.3"></a>

When we solve a PDE by converting it to a difference equation, we hope
that the solution of the latter is a good approximation to the solution
of the former. If the difference-equation solution diverges, then we
know we have a bad approximation, but if it converges, then we may feel
confident that we have a good approximation to the PDE. The *von Neumann
stability analysis* is based on the assumption that eigenmodes of the
difference equation can be expressed as

$$\tag*{20.18} T_{m,j} = \xi(k)^j e^{ikm\Delta x},$$

where $x=m \Delta x$ and $t=j \Delta t$, and $i=\sqrt{-1}$ is the imaginary
number. The constant $k$ in (20.18) is an unknown wave vector
($2\pi/\lambda$), and $\xi(k)$ is an unknown complex function. View (20.18) as
a basis function that oscillates in space (the exponential) with an amplitude or
*amplification factor* $\xi(k)^j$ that increases by a power of $\xi$ for each
time step. If the general solution to the difference equation can be expanded in
terms of these eigenmodes, then the general solution will be stable if the
eigenmodes are stable. Clearly, for an eigenmode to be stable, the amplitude
$\xi$ cannot grow in time $j$, which means $|xi(k)|<1$ for all values of the
parameter $k$ \[[Press et al.(94)](BiblioLinked.html#press), [Ancona(02)](BiblioLinked.html#ancona)\].

Application of a stability analysis is more straightforward than it
might appear. We just substitute the expression (20.18) into the
difference equation (20.17):

$$ \tag*{20.19}
\xi^{j+1} e^{ikm\Delta x}  =   \xi^{j+} e^{ikm\Delta x} 
   +\eta
\left[\xi^{j} e^{ik(m+1)\Delta x} + \xi^{j+} e^{ik(m-1)\Delta x}
-2 \xi^{j+} e^{ikm\Delta x}\right] .$$

After canceling some common factors, it is easy to solve for $\xi$:

$$\tag*{20.20}
\xi(k) = 1 + 2\eta [\cos(k\Delta x)-1] .$$

In order for $|xi(k)|<1$ for all possible $k$ values, we must have

$$\tag*{20.21}
\eta  = \frac{K \Delta t}{C\rho \Delta x^2}
<\frac{1}{2}.$$

This equation tells us that if we make the time step $\Delta t$ smaller,
we will always improve the stability, as we would expect. But if we
decrease the space step $\Delta x$ without a simultaneous quadratic
*increase* in the time step, we will worsen the stability. The lack of
space-time symmetry arises from our use of stepping in time, but not in
space.

In general, you should perform a stability analysis for every PDE you
have to solve, although it can get complicated \[Press et al.(94)\]. Yet
even if you do not, the lesson here is that you may have to try
different *combinations* of $\Delta x$ and $\Delta t$ variations until a
stable, reasonable solution is obtained. You may expect, nonetheless,
that there are choices for $\Delta x$ and $\Delta t$ for which the
numerical solution fails and that simply decreasing an individual
$\Delta x$ or $\Delta t$, in the hope that this will increase precision,
may not improve the solution.

[**Listing 20.1  EqHeat.py**](http://www.science.oregonstate.edu/~rubin/Books/CPbook/Codes/PythonCodes/EqHeat.py) solves the 1-D space heat equation on a lattice by
leapfrogging (time-stepping) the initial conditions forward in time. You will need
to adjust the parameters to obtain a solution like those in the figures.

In [ ]:
# EqHeat.py, Notebook Version

#from __future__ import division,print_function
from IPython.display import IFrame
from numpy import *
import numpy as np
import matplotlib.pylab as p
from mpl_toolkits.mplot3d import Axes3D ;

Nx = 101;        Nt = 9000;     Dx = 0.01;     Dt = 0.3                                                              
KAPPA = 210.;    SPH = 900.;    RHO = 2700.   # conductivity, specific heat, density                                                      
T = zeros( (Nx, 2), float);  Tpl = zeros( (Nx, 31), float)  
                                     
print("Working, wait for figure after count to 30")

for ix in range (1, Nx - 1):
    T[ix, 0] = 100.0;                 # initial temperature
T[0,0] = 0.0 ;   T[0,1] = 0.          # first and last points at 0
T[Nx-1,0] = 0. ; T[Nx-1,1] = 0.0
cons = KAPPA/(SPH*RHO)*Dt/(Dx*Dx);                                         # constant
m = 1           # counter for rows, one every 300 time steps

for t in range (1, Nt):                                              # time iteration
    for ix in range (1, Nx - 1):                                  # Finite differences
        T[ix, 1] = T[ix, 0] +  cons*(T[ix + 1, 0] + T[ix - 1, 0] - 2.0*T[ix, 0])                                                        
    if t%300 == 0 or t == 1:  # for t = 1 and every 300 time steps
        for ix in range (1, Nx - 1, 2): Tpl[ix, m] = T[ix, 1]   
        print(m)   
        m = m + 1           # increase m every 300 time steps
    for ix in range (1, Nx - 1):  T[ix, 0] = T[ix, 1]            # 100 positons at t=m
x = list(range(1, Nx - 1, 2))                                    # plot every other x point
y = list(range(1, 30))                                        # every 10 points in y (time)
X, Y = p.meshgrid(x, y)      # grid for position and time

def functz(Tpl):             # Function returns temperature
    z = Tpl[X, Y]       
    return z

Z = functz(Tpl)              
fig = p.figure()                                                      # create figure
ax = Axes3D(fig)                                                      # plots axis
ax.plot_wireframe(X, Y, Z, color = 'r')                               # red wireframe
ax.set_xlabel('Position')                                                # label axes
ax.set_ylabel('time')
ax.set_zlabel('Temperature')
p.show()                  # shows figure, close Python shell
print("finished")         

Working, wait for figure after count to 30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


### EqHeatAnimate.py, Notebook Version

In [ ]:
# EqHeatAnimate.py, Notebook Version

from __future__ import division,print_function
from ivisual import *
#from IPython.display import IFrame
#import random
#from numpy import *

scene=canvas(title="Heat Equation")
scene.width=500
scene.height=300
scene.range=100

bar = curve( pos = [( - 100,  - 20), (100,  - 20)], color = color.magenta ) 
ball1 = sphere( pos = (100,  - 20,0), color = color.blue, radius = 4 ) 
ball2 = sphere( pos = ( - 100,  - 20,0), color = color.blue, radius = 4 )
tempe = curve(color = color.red,radius=1)
Nx = 101                                         # Grid points in x
Dx = 0.01414                                     # x increment
Dt = 0.6                                         # t increment
KAPPA = 210.                                     # Thermal conductivity
SPH = 900.                                       # Specific heat
RHO = 2700.                                                                 # Density
T = zeros( (Nx, 2), float)                       # Temp @ first 2 times
for ix in range (1, Nx - 1):                         # Initial temperature in the bar
     T[ix, 0] = 100.0     
T[0,0] = 0.0                                                  # Ends of bar at T = 0
T[0,1] = 0.                          
T[Nx - 1, 0] = 0.
T[Nx - 1, 1] = 0.0
cons = KAPPA/(SPH*RHO)*Dt/(Dx*Dx);                      # Constant combo in algorthim
'''
for i in range (0, Nx - 1):            
    tempe.x[i] =  2.0*i  - 100.0                                         # Scaled x's
    tempe.y[i] = 0.8*T[i, 0] - 20.0                               # Scaled y's (Temp)
'''
for a in range(0,2):
    #print(a)
    print(" cons  ",cons)
    rate(50)# Times loop
    for ix in range (1, Nx - 1):                                  # Finite differences
        T[ix,1] = T[ix,0] + cons*(T[ix +1,0]+T[ix-1,0]-2.0*T[ix,0]) 
     
    for i in range (0, Nx):
        xx = 2.0*i - 100.0                        # Scale 0<x<100 -> -100<x<100
        tempe.append(pos= (xx,0.6*T[i,1]-20.0,0),color=(1,0,0))
        print(xx,0.6*T[i,1]-20)
       
    for ix in range (1, Nx - 1):
        xx = 2.0*i - 100.0                        # Scale 0<x<100 -> -100<x<100
        tempe.append(pos= (xx,0.6*T[ix,1]-20.0,0),color=(0,0,0))
        print(xx,T[ix,1])
        T[ix, 0] = T[ix, 1]                           # Row of 100 positions at t = m

### 20.2.4  Heat Equation Implementation

Recall that we want to solve for the temperature distribution within an
aluminum bar of length $L=1$ m subject to the boundary and initial
conditions

$$\tag*{20.22} T(x=0,t) = T(x=L,t) = 0 {\rm K}, \hspace{6ex} T(x,t=0) =
 100 \mbox{K}.$$

The thermal conductivity, specific heat, and density for Al are

$$\tag*{20.23} K =237 \mbox{W/(mK)},\quad C = 900 \mbox{{J/(kg K)}}, \quad
\rho= 2700 \mbox{kg/m}^3.$$

1.  Write or modify `EqHeat.py` in Listing 20.1 to solve the
    heat equation.

2.  Define a 2-D array `T[101,2]` for the temperature as a function of
    space and time. The first index is for the $100$ space divisions of
    the bar, and the second index is for present and past times (because
    you may have to make thousands of time steps, you save memory by
    saving only two times).

3.  For time $t=0$ ( *j = <span>1</span>*), initialize `T` so that all
    points on the bar except the ends are at 100 K. Set the temperatures
    of the ends to 0 K.

4.  Apply (20.14) to obtain the temperature at the next time step.

5.  Assign the present-time values of the temperature to the past
    values: `T[i,1] = T[i,2], i = 1, . . . , 101.`

6.  Start with 50 time steps. Once you are confident the program is
    running properly, use thousands of steps to see the bar cool
    smoothly with time. For approximately every 500 time steps, print
    the time and temperature along the bar.

## 20.3  Assessment and Visualization <a id="20.3"></a>

1.  Check that your program gives a temperature distribution that varies
    smoothly along the bar and agrees with the boundary conditions, as
    in Figure ref<span>heat-1.png</span>.

2.  Check that your program gives a temperature distribution that varies
    smoothly with time and reaches equilibrium. You may have to vary the
    time and space steps to obtain stable solutions.

3.  Compare the analytic and numeric solutions (and the wall times
    needed to compute them). If the solutions differ, suspect the one
    that does not appear smooth and continuous.

4.  Make a surface plot of temperature *versus* position *versus* time.

5.  Plot the *isotherms* (contours of constant temperature).

6.  **Stability test:** Verify the stability condition (20.21) by
    observing how the temperature distribution diverges if
    $\eta>{\frac {1}{4}}$.

7.  **Material dependence:** Repeat the calculation for iron. Note that
    the stability condition requires you to change the size of the
    time step.

8.  **Initial sinusoidal distribution $\sin(\pi x/ L)$:** Compare to the
    analytic solution,

    $$\tag*{20.24}
    T(x,t)= \sin (\pi x/L) e^{ -\pi^2 Kt/(L^2 C\rho)} .$$

9.  **Two bars in contact:** Two identical bars 0.25 m long are placed
    in contact along one of their ends with their other ends kept at
    0 K. One is kept in a heat bath at 100 K, and the other at 50 K.
    Determine how the temperature varies with time and
    location (Figure 20.4).

10. **Radiating bar (Newton’s cooling):** Imagine now that instead of
    being insulated along its length, a bar is in contact with an
    environment at a temperature $T_e$. Newton’s law of
    cooling (radiation) says that the rate of temperature change as a
    result of radiation is

    $$\tag*{20.25}
    \frac{\partial T} { \partial t}=-h(T-T_e),$$

    where $h$ is a positive constant. This leads to the modified heat
    equation

    $$\tag*{20.26}
    \frac{\partial T(x,t)}{\partial t}=  \frac{K}{C\rho}
    \frac{\partial^2 T}{\partial^2 x}-h T(x,t).$$

    Modify the algorithm to include Newton’s cooling and compare the
    cooling of a radiating bar with that of the insulated bar.

![image](Figs/Fig20_4.png)

**Figure 20.4** Temperature *versus* position and time when two bars at
differing temperatures are placed in contact at $\textit{t}=\text{0}$. The
projected contours show the isotherms.

## 20.4  Improved Heat Flow: Crank-Nicolson Method <a id="20.4"></a>

The Crank-Nicolson method \[[Crank & Nicolson(46)](BiblioLinked.html#CN)\] provides a higher
degree of precision for the heat equation (20.3) than the simple
leapfrog method we have just discussed. This method calculates the time
derivative with a central-difference approximation, in contrast to the
forward-difference approximation used previously. In order to avoid
introducing error for the initial time step where only a single time
value is known, the method uses a *split time step*,\[*Note:* In §22.2.1
we develop a different split-time algorithm for the solution of the
Schrödinger equation. There the real and imaginary parts of the wave
function are computed at times that differ by $\Delta t/2$.\] so that
time is advanced from time $t$ to $ t+ \Delta t/2$:

$$\tag*{20.27} {\frac{\partial T}{\partial t}}\left(x,t+ \frac{\Delta t}{2}\right)
\simeq {\frac{T(x,t+\Delta t)-T(x,t)}{\Delta t}+ O(\Delta t^2)}.$$

Yes, we know that this looks just like the forward-difference approximation for
the derivative at time $t+\Delta t$, for which it would be a bad approximation;
regardless, it is a better approximation for the derivative at time $t+ \Delta t/2$,
although it makes the computation more complicated. Likewise, in (20.14) we
gave the central-difference approximation for the second space derivative for
time $t$. For $t= t+ \Delta t/2$, that becomes 

$$\begin{align}
 & 2 (\Delta x)^2  {\frac{\partial^2 T}{\partial
x^2}}\left(x, t+\frac{\Delta t}{2}\right)   \simeq  + \left[ T(x-\Delta x,  t)-2T(x,  t)+T(x+\Delta x,  t)
\right]   \\
 & + [T(x-\Delta x,  t+\Delta t) -  2T(x,  t +\Delta t)
+  T(x+\Delta x,  t+\Delta t)]
     +O(\Delta x^2).\tag*{20.28}\end{align}$$ 
     
In terms of these expressions, the heat difference equation is 

$$\begin{align} T_{i,j+1}-T_{i,j} &=
\frac{\eta} {2}
\left[T_{i-1,j+1}-2T_{i,j+1}+T_{i+1,j+1} +
T_{i-1,j}-2T_{i,j}+T_{i+1,j}\right],\\
 x &=i\Delta x, \  t=j\Delta t, \
\eta=\frac{K\Delta t}{C\rho \Delta x^2}.\tag*{20.29}\end{align}$$ 

We
group together terms involving the same temperature to obtain an
equation with future times on the LHS and present times on the RHS:

$$\tag*{20.30}
 -T_{i-1,
j+1}+\left(\frac{2}{\eta}+2\right)T_{i,  j+1}-T_{i+1,  j+1} =
T_{i-1,  j} + \left(\frac{2}{\eta}-2\right)T_{i,  j} + T_{i+1,
j}.$$

This equation represents an *implicit* scheme for the temperature
$T_{i,j}$, where the word “implicit” means that we must solve
simultaneous equations to obtain the full solution for all space. In
contrast, an *explicit* scheme requires iteration to arrive at the
solution. It is possible to solve (20.30) simultaneously for all unknown
temperatures ($1
\le i \le N$) at times $j$ and $j + 1$. We start with the initial
temperature distribution throughout all of space, the boundary
conditions at the ends of the bar for all times, and the approximate
values from the first derivative:

$$\tag*{20.31}
\begin{array}{lll}
T_{i, 0}, \mbox{known}, \quad&T_{0, j}, \mbox{known}, \quad &T_{N, j},
\mbox{known}, \\
T_{0, j+1} = T_{0, j}=0, &T_{N, j+1}= 0,& T_{N, j}= 0.
\end{array}$$

We rearrange (20.30) so that we can use these known values of $T$ to
step the $j=0$ solution forward in time by expressing (20.30) as a set
of simultaneous linear equations (in matrix form):

$$\begin{align} &\begin{pmatrix} 
\big(\frac{2}{\eta}+2\big) & -1 & & & & \\
 -1 &\big(\frac{2}{\eta}+2\big) & -1 & & & \\
 & -1 & \big(\frac{2}{\eta}+2\big) & -1 & & \\
 & & \ddots & \ddots & \ddots &  & \\
 & & & -1 &\big( \frac{2}{\eta}+2\big) &-1 &      \\
 & & & &-1& \big(\frac{2}{\eta}+2\big) \\
 \end{pmatrix}
\begin{pmatrix}
 T_{1,j+1}\\
 T_{2,j+1}\\
 T_{3,j+1)}\\
 \vdots \\
 T_{n-2,j+1}\\
 T_{n-1,j+1}
 \end{pmatrix}     \\
 & \quad =\begin{pmatrix}
 T_{0,j+1}+T_{0,j}+\big(\frac{2}{\eta}-2\big)T_{1,j}+T_{2,j}\\
 T_{1,j}+\big(\frac{2}{\eta}-2\big)T_{2,j}+T_{3,j}\\
 T_{2,j}+\big(\frac{2}{\eta}-2\big)T_{3,j}+T_{4,j}\\
 \vdots \\
 T_{n-3,j}+\big(\frac{2}{\eta}-2\big)T_{n-2,j}+T_{n-1,j}\\
 T_{n-2,j}+\big(\frac{2}{\eta}-2\big)T_{n-1,j}+T_{n,j}+T_{n,j+1}
 \end{pmatrix}. \tag*{20.32}\end{align}$$

Observe that the $T$’s on the RHS are all at the present time $j$ for
various positions, and at future time $j+1$ for the two ends (whose $T$s
are known for all times via the boundary conditions). We start the
algorithm with the $T_{i,j=0}$ values of the initial conditions, then
solve a matrix equation to obtain $T_{i,j=1}$. With that we know all the
terms on the RHS of the equations ($j=1$ throughout the bar and $j=2$ at
the ends) and so can repeat the solution of the matrix equations to
obtain the temperature throughout the bar for $j=2$. So again, we
time-step forward, only now we solve matrix equations at each step. That
gives us the spatial solution at all locations directly.

Not only is the Crank-Nicolson method more precise than the low-order
time-stepping method, but it also is stable for all values of $\Delta t$ and
$\Delta x$. To prove that, we apply the von Neumann stability analysis
discussed in §20.2.3 to the Crank-Nicolson algorithm by substituting (20.17)
into (20.30). This determines an amplification factor 

$$\begin{align} \tag*{20.33}
\xi(k) = \frac{1-2\eta    \sin^2( {k\Delta x/2})}{1+2\eta
\sin^2({k\Delta x/2})}.\end{align}$$ 

Because $\sin^2()$ is
positive-definite, this proves that $|xi|
\leq 1$ for all $\Delta t$, $\Delta x$, and $k$.

### 20.4.1  Solution of Tridiagonal Matrix Equations $\odot$<a id="20.4.1"></a>

The Crank-Nicolson equations (20.32) are in the standard
$[A] \mathbf{x} = \mathbf{b}$ form for linear equations, and so we can
use our previous methods to solve them. Nonetheless, because the
coefficient matrix $[A]$ is tridiagonal (zero elements except for the
main diagonal and two diagonals on either side of it),

$$\begin{pmatrix}
 d_1 & c_1 & 0   & 0    & \cdots & \cdots & \cdots  & 0 &\\
 a_2 & d_2 & c_2 & 0    & \cdots & \cdots & \cdots  & 0  &\\
 0 & a_3 & d_3   & c_3  &\cdots &\cdots  &\cdots   & 0 &\\
 \cdots & \cdots & \cdots   & \cdots    & \cdots & \cdots & \cdots  & \cdots
 &\\
 0 & 0 & 0   & 0    &\cdots  & a_{N-1}  & d_{N-1}    & c_{N-1}
 &\\
 0 & 0 & 0   & 0    & \cdots & 0    & a_{N}  & d_{N} &
 \end{pmatrix}
\begin{pmatrix}
 x_1\\
 x_2\\
 x_3\\
 \ddots \\
 x_{N-1}\\
 x_N
 \end{pmatrix} =\begin{pmatrix}
 b_1\\
 b_2\\
 b_3\\
 \ddots \\
 b_{N-1}\\
 b_N
 \end{pmatrix},\tag*{20.34}$$

a more robust and faster solution exists that makes this implicit method
as fast as an explicit one. Because tridiagonal systems occur
frequently, we now outline the specialized technique for solving them
\[[Press et al.(94)](BiblioLinked.html#press)\]. If we store the matrix elements $a_{i,j}$ using
both subscripts, then we will need $N^2$ locations for elements and
$N^2$ operations to access them. However, for a tridiagonal matrix, we
need to store only the vectors $\left\{d_i\right\}_{i=1,N}$,
$\left\{c_i\right\}_{i=1,N}$, and $\left\{a_i\right\}_{i=1,N}$, along,
above, and below the diagonals. The single subscripts on $a_i$, $d_i$,
and $c_i$ reduce the processing from $N^2$ to ($3N-2$) elements.

We solve the matrix equation by manipulating the individual equations until the
coefficient matrix is *upper triangular* with all the elements of the main
diagonal equal to $1$. We start by dividing the first equation by $d_1$, then
subtract $a_2$ times the first equation, 

$$\begin{align} &\begin{pmatrix} 1 &
\frac{c_1}{d_1} & 0 & 0 &\cdots &\cdots &\cdots &0 &\\
 0 & d_2-\frac{a_2c_1}{d_1} & c_2 & 0    & \cdots &\cdots    & \cdots & 0 &\\
   0 & a_3 & d_3   & c_3  &\cdots &\cdots  &\cdots        & 0 &\\
   \cdots & \cdots & \cdots   & \cdots    & \cdots & \cdots  & \cdots & \cdots&\\
   0 & 0 & 0   & 0    & \cdots & a_{N-1}  & d_{N-1}    & c_{N-1} &\\
   0 & 0 & 0   & 0    & \cdots & 0    & a_{N}  & d_{N} &
  \end{pmatrix}
\begin{pmatrix}
  x_1\\
  x_2\\
  x_3\\
  \ddots \\
  \cdot \\
  x_N
  \end{pmatrix} 
      =\begin{pmatrix}
  \frac{b_1}{d_1}\\
  b_2-\frac{a_2b_1}{d_1}\\
  b_3\\
  \ddots \\
  \cdot\\
  b_N
  \end{pmatrix},&\tag*{20.35}\end{align}$$

and then dividing the second equation by the second diagonal element,

$$\begin{align} 
&\begin{pmatrix} 
1 & \frac{c_1}{d_1} & 0 & 0 &\cdots &\cdots
&\cdots &0 &\\
 0 & 1 &\frac{ c_2}{d_2-a_2\frac{c_1}{a_1}} & 0  & \cdots & & \cdots    & 0  &\\
   0 & a_3 & d_3   & c_3  &\cdots &\cdots  &\cdots        & 0 &\\
   \cdots & \cdots & \cdots   & \cdots    & \cdots & \cdots  & \cdots & \cdots&\\
   0 & 0 & 0   & 0    &    & a_{N-1}  & d_{N-1}    & c_{N-1} &\\
   0 & 0 & 0   & 0    & \cdots & 0    & a_{N}  & d_{N} &
  \end{pmatrix}
\begin{pmatrix}
  x_1\\
  x_2\\
  x_3\\
  \ddots \\
  \cdot \\
  x_N
  \end{pmatrix}
   =\begin{pmatrix}
  \frac{b_1}{d_1}\\
  \frac{b_2-a_2\frac{b_1}{d_1}}{d_2-a_2\frac{c_1}{d_1}}\
  b_3\\
  \ddots \\
  \cdot \\
  b_N
  \end{pmatrix}.\tag*{20.36}\end{align}$$

Assuming that we can repeat these steps without ever dividing by zero,
the system of equations will be reduced to upper triangular form,

$$\tag*{20.37}
\begin{pmatrix}
1 & h_1 & 0 & 0 & \cdots & 0 &\\
 0 & 1 & h_2     & 0  & \cdots    & 0    &\\
 0 &  0 & 1 & h_3& \cdots    & 0 &\\
 0& \cdots & \cdots     & \ddots    & \ddots & \cdots &\\
 0 & 0 & 0   & 0    & \cdots & \cdots       &\\
 0 & 0 & 0   &   \cdots &    0 & 1 &
\end{pmatrix}
\begin{pmatrix}
 x_1\\
 x_2\\
 x_3\\
 \ddots \\
 \cdot \\
 x_N
 \end{pmatrix} =\begin{pmatrix}
 p_1\\
 p_2\\
 p_3\\
 \ddots \\
 \cdot \\
 p_N
 \end{pmatrix},$$

where $h_1 = {c_1}/{d_1}$ and $p_1 =  {b_1}/{d_1}$. We then recur for
the others elements:

$$\tag*{20.38} h_i=\frac{c_i}{d_i-a_i h_{i-1}},\quad
p_i=\frac{b_i-a_ip_{i-1}}{d_i-a_i h_{i-1}}.$$

Finally, back substitution leads to the explicit solution for the
unknowns:

$$\tag*{20.39} x_i=p_i-h_ix_{i-1}; \quad i=n-1,n-2,\ldots ,1 , \quad x_N=p_N.$$

In Listing 20.2 we give the program `HeatCNTridiag.py` that solves the
heat equation using the Crank-Nicolson algorithm via a triadiagonal
reduction.

[**Listing 20.2  HeatCNTridiag.py**](http://www.science.oregonstate.edu/~rubin/Books/CPbook/Codes/PythonCodes/HeatCNTridiag.py) is the complete program for solution of the
heat equation in one space dimension and time via the Crank-Nicolson method.
The resulting matrix equations are solved via a technique specialized to
tridiagonal matrices.

In [ ]:
# HeatCrankNicolson.py, Notebook Version

from __future__ import division,print_function
from IPython.display import IFrame
from numpy import *

import numpy as np
import matplotlib.pylab as p
from mpl_toolkits.mplot3d import Axes3D ;


Max = 51; n   = 50;   m = 50
Ta  = zeros( (Max), float );  Tb  = zeros( (Max), float ); Tc  = zeros((Max), float)
Td  = zeros( (Max), float );  a   = zeros( (Max), float ); b   = zeros( (Max), float)
c   = zeros( (Max), float );  d   = zeros( (Max), float ); x   = zeros( (Max), float)
t   = zeros( (Max, Max), float )

def Tridiag(a, d, c, b, Ta, Td, Tc, Tb, x, n):                # Define Tridiag method
    Max = 51
    h = zeros( (Max), float )
    p = zeros( (Max), float )
    for i in range(1,n+1):
        a[i] = Ta[i]
        b[i] = Tb[i]
        c[i] = Tc[i]
        d[i] = Td[i]
    h[1] = c[1]/d[1]
    p[1] = b[1]/d[1]
    for i in range(2,n+1):
        h[i] = c[i] / (d[i]-a[i]*h[i-1])
        p[i] = (b[i] - a[i]*p[i-1]) / (d[i]-a[i]*h[i-1])
    x[n] = p[n]
    for i in range( n - 1, 1,-1 ): x[i] = p[i] - h[i]*x[i+1]

width = 1.0; height = 0.1; ct = 1.0                                 # Rectangle W & H
for i in range(0, n):   t[i,0] = 0.0                                     # Initialize
for i in range( 1, m):  t[0][i] = 0.0
h  = width  / ( n - 1 )                            # Compute step sizes and constants
k  = height / ( m - 1 )
r  = ct * ct * k / ( h * h )

for j in range(1,m+1):
    t[1,j] = 0.0
    t[n,j] = 0.0                                                               # BCs
for i in range( 2, n):   
    t[i][1] = sin( pi * h *i)                                                   # ICs
for i in range(1, n+1):  
    Td[i] = 2. + 2./r
Td[1] = 1.
Td[n] = 1.
for i in range(1,n ): Ta[i] = -1.0;      Tc[i] = -1.0;                 # Off diagonal
Ta[n-1] = 0.0;   Tc[1] = 0.0; Tb[1] = 0.0; Tb[n] = 0.0

for j in range(2,m+1):
    for i in range(2,n): 
        Tb[i] = t[i-1][j-1] + t[i+1][j-1] + (2/r-2) * t[i][j-1]
    Tridiag(a, d, c, b, Ta, Td, Tc, Tb, x, n)                        # Solve system
    for i in range(1, n+1):  
        t[i][j] = x[i]
print("Finished")
x = list(range(1, m+1))                                          # Plot every other x point
y = list(range(1, n+1))                                               # every other y point
X, Y = p.meshgrid(x,y)               

def functz(t):                                           # Function returns potential
    z = t[X, Y]                 
    return z
Z = functz(t)                          
fig = p.figure()                                                      # Create figure
ax = Axes3D(fig)                                                      # plots axes
ax.plot_wireframe(X, Y, Z, color= 'r')                                # red wireframe
ax.set_xlabel('t')                                                       # label axes
ax.set_ylabel('x')
ax.set_zlabel('T')
p.show()             

### 20.4.2  Crank-Nicolson Implementation, Assessment<a id="20.4.2"></a>

1.  Write a program using the Crank-Nicolson method to solve for the
    heat flow in the metal bar of § 20.1 for at least 100 time steps.

2.  Solve the linear system of equations (20.32) using either Matplotlib
    or the special tridiagonal algorithm.

3.  Check the stability of your solution by choosing different values
    for the time and space steps.

4.  Construct a contoured surface plot of temperature
    <span>*versus*</span> position and versus time.

5.  Compare the implicit and explicit algorithms used in this chapter
    for relative precision and speed. You may assume that a stable
    answer that uses very small time steps is accurate.